In [3]:
from spp_net_f import *
from spp_layer_f import *
from load_mnist import *
import os
import time
import input_data
import tensorflow as tf
import numpy as np



train_size = 3060
batch_size = 50
max_epochs =2
num_class = 10
eval_frequency = 100
max_steps = 10000

     



In [4]:
class data_label_data():
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def next_batch(self, batch_size):
        label_len = len(self.label)
        shuffle_index = np.arange(label_len)
        np.random.shuffle(shuffle_index)
        batch_index = shuffle_index[:batch_size]
        data_batch = self.data[batch_index]
        label_batch = self.label[batch_index]
        return [data_batch, label_batch]
class load_data(data_label_data):
    def __init__(self, train, test):
        self.train = train
        self.test = test

def train():
    global_step = tf.Variable(0, trainable=False)
    spp_net = SPPnet()
    spp_net.set_lr(0.0001, batch_size, train_size)
    
# load data
    print('load data')
    train_data = loadImageSet("train-images.idx3-ubyte")
    train_label = loadLabelSet("train-labels.idx1-ubyte")
    test_data = loadImageSet("t10k-images.idx3-ubyte")
    test_label = loadLabelSet("t10k-labels.idx1-ubyte")

    train_part = data_label_data(train_data, train_label)
    test_part = data_label_data(test_data, train_label)
    mnist = load_data(train_part, test_part)
#     mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  
    x_mag = tf.placeholder("float", shape=[None, 784])
    train_data = tf.reshape(x_mag, [-1,28,28,1])
    train_label = tf.placeholder("float", shape=[None, 10])
    keep_prob = tf.placeholder("float")
    num_class = 10
    print("load done")


# train
    print('train')
 
    logits = spp_net.inference(train_data, True, num_class, tp='spp')
    loss, accuracy, opt  = spp_net.train(logits, global_step, train_label)
    print('train done')
    
# evaluation
#    eval_logits = spp_net.inference(valid_data, False, num_class)
#    eval_accuracy = spp_net.loss(eval_logits, valid_label)
    with tf.Session() as sess:
        saver = tf.train.Saver(tf.all_variables())
        init = tf.initialize_all_variables()
        sess.run(init)        
        start_time = time.time()
    #    print((FLAGS.max_epochs * train_size) // batch_size)
        for step in range(max_steps):
            batch = mnist.train.next_batch(batch_size)
            opt.run( feed_dict={ x_mag:batch[0], train_label: batch[1], keep_prob: 0.5 } )
            if step % eval_frequency ==0:
                stop_time = time.time() - start_time
                start_time = time.time()
                loss_value=loss.eval(feed_dict={x_mag:batch[0], train_label: batch[1], keep_prob: 1.0})
                accu = accuracy.eval(feed_dict={x_mag:batch[0], train_label: batch[1], keep_prob: 1.0})
                print('epoch: %.2f , %.2f ms' % (step * batch_size /train_size,
                    1000 * stop_time / eval_frequency)) 
                print('train loss: ', loss_value) 
                print('train accu: ', accu)         


if __name__ == '__main__':
    train()




load data
load image set train-images.idx3-ubyte
head (2051, 60000, 28, 28)
load imgs finished
load label set train-labels.idx1-ubyte
head, (2049, 60000)
load label finished
load image set t10k-images.idx3-ubyte
head (2051, 10000, 28, 28)
load imgs finished
load label set t10k-labels.idx1-ubyte
head, (2049, 10000)
load label finished
load done
train
pool1.shape (?, 14, 14, 6)
conv3.shape (?, 14, 14, 16)
bin_strides [4, 7, 14]
bin_filters [5, 7, 14]
pooled_out [<tf.Tensor 'SPP/Reshape:0' shape=(?, 144) dtype=float32>, <tf.Tensor 'SPP/Reshape_1:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'SPP/Reshape_2:0' shape=(?, 16) dtype=float32>]
(?, 224)
numH 224
inference
train done
Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
epoch: 0.00 , 6.40 ms
train loss:  115.005
train accu:  0.04
epoch: 1.63 , 117.00 ms
train loss:  111.864
train accu:  0.24
epoch: 3.27 , 118.09 ms
train loss:  98.8913
train accu

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "d:\python\lib\site-packages\IPython\core\interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-4cc30b2a10d8>", line 73, in <module>
    train()
  File "<ipython-input-4-4cc30b2a10d8>", line 60, in train
    opt.run( feed_dict={ x_mag:batch[0], train_label: batch[1], keep_prob: 0.5 } )
  File "d:\python\lib\site-packages\tensorflow\python\framework\ops.py", line 1552, in run
    _run_using_default_session(self, feed_dict, self.graph, session)
  File "d:\python\lib\site-packages\tensorflow\python\framework\ops.py", line 3776, in _run_using_default_session
    session.run(operation, feed_dict)
  File "d:\python\lib\site-packages\tensorflow\python\client\session.py", line 778, in run
    run_metadata_ptr)
  File "d:\python\lib\site-packages\tensorflow\python\client\session.py", line 982, in _run
    feed_dict_string, options, run_metadata)
  File "d:\python\lib\site-packages\ten

KeyboardInterrupt: 